# EEL891 - 2023.02 - Trabalho 2

Este trabalho envolve utilizar técnicas de regrssão para estimar o preço de um imóvel a partir de suas características.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import GridSearchCV, cross_val_score

## 1. Lendo e organizando dados

Aqui estou lendo os dados de teste e treinamento dos arquivos csv e depois juntando-os em um só arquivo "imovel_df" para que possamos fazer o pré-processamento de uma vez só.

"split_index" é uma variável para guardar o ponto de junção dos conjuntos de dados (corresponde ao primeiro índice de 'imovel_df' que faz parte do test set) para que depois eles possam ser separados de novo. "Id" guarda os IDs dos imóveis que terão seu campo 'preco' previsto pelo modelo; estes IDs farão parte do arquivo de saída que será enviado ao Kaggle e foram separados para facilitar.

In [2]:
train = pd.read_csv('conjunto_de_treinamento.csv')
test = pd.read_csv('conjunto_de_teste.csv')

In [5]:
# Salvar os Ids para auxiliar na criação do arquivo que será enviado ao Kaggle
id = test['Id']

# Criando um DF novo com o train e o test
imovel_df = pd.concat([train, test], ignore_index=True)

In [7]:
# Para auxiliar no momento separar o DF credito_df em train e test
split_index = len(train)

In [6]:
imovel_df

,Id,tipo,bairro,tipo_vendedor,quartos,suites,vagas,area_util,area_extra,diferenciais,...,estacionamento,piscina,playground,quadra,s_festas,s_jogos,s_ginastica,sauna,vista_mar,preco
0,2000,Casa,Imbiribeira,Imobiliaria,3,3,5,223,167,piscina e copa,...,0,1,0,0,0,0,0,0,0,1000000.0
1,2001,Apartamento,Casa Amarela,Imobiliaria,4,4,2,157,0,piscina e churrasqueira,...,0,1,0,0,0,0,0,0,0,680000.0
2,2002,Apartamento,Encruzilhada,Imobiliaria,3,1,0,53,0,nenhum,...,0,0,0,0,0,0,0,0,0,450000.0
3,2003,Apartamento,Boa Viagem,Imobiliaria,4,3,2,149,0,piscina e churrasqueira,...,0,1,0,0,0,0,0,0,0,1080000.0
4,2004,Apartamento,Rosarinho,Imobiliaria,2,1,1,54,0,piscina e churrasqueira,...,0,1,0,0,0,0,0,0,0,350000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6678,1995,Apartamento,Poco da Panela,Imobiliaria,2,1,2,100,0,nenhum,...,0,0,0,0,0,0,0,0,0,NaN
6679,1996,Apartamento,Madalena,Imobiliaria,3,1,1,90,0,nenhum,...,0,0,0,0,0,0,0,0,0,NaN
6680,1997,Apartamento,Boa Viagem,Imobiliaria,3,1,1,73,0,piscina e salao de festas,...,0,1,0,0,1,0,0,0,0,NaN
6681,1998,Apartamento,Torre,Imobiliaria,4,3,2,112,2810,piscina e frente para o mar,...,0,1,0,0,0,0,0,0,1,NaN


## 2. Preparação dos dados
Análise e preparação dos dados para treinamento

### 2.1. Valores Nulos
Antes de tudo, é fundamental lidar com valores ausentes no conjunto de dados. Aqui, vemos que todas as colunas estão preenchidas e não há nulos.

In [8]:
imovel_df.isnull().sum()

Id                   0
tipo                 0
bairro               0
tipo_vendedor        0
quartos              0
suites               0
vagas                0
area_util            0
area_extra           0
diferenciais         0
churrasqueira        0
estacionamento       0
piscina              0
playground           0
quadra               0
s_festas             0
s_jogos              0
s_ginastica          0
sauna                0
vista_mar            0
preco             2000
dtype: int64

### 2.2. Modificação de Features
Aqui fiz uma breve análise de como 'otimizar' algumas features para melhores previsões. 

In [11]:
imovel_df['churrasqueira'].unique()

array([0, 1], dtype=int64)